<img src="./include/img/fabric_logo.png" width="500"/>

# DHCP
```
Created: 2023-02-01
Last Modified: April 10, 2023
Devin Lane
Clemson University
ddlane@clemson.edu
```

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak 
</div>

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/Users/devinlane/Documents/fabric/src/fabric_config/id_token.json
Project ID,7bc05490-ba40-45ba-99dc-b011680825d2
Bastion Username,ddlane_0000006976
Bastion Private Key File,/Users/devinlane/Documents/fabric/src/fabric_config/bastion
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/Users/devinlane/Documents/fabric/src/fabric_config/slice_key.pub
Slice Private Key File,/Users/devinlane/Documents/fabric/src/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
# Clean up log directory to avoid large files
import os
for item in os.scandir(os.path.join(os.getcwd(), 'logs')):
    if '.' != item.name[0]:
        os.remove(item.path)

# Submit the Slice

In [3]:
slice_name = 'DHCPSlice'

site = 'CLEM'
print(f"Site: {site}")

node1_name = 'Server'
node2_name = 'Client1'
node3_name = 'Client2'
network_name='DHCP-demo-net'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
node3_nic_name = 'nic3'
image = 'default_ubuntu_22'

Site: CLEM


Here we will add three nodes to an L2 network with a subnet of 10.0.0.0/24. We don't care which IP address the nodes get so we will set the mode for each interface to 'auto'.

In [4]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
subnet = IPv4Network("10.0.0.0/24")
available_ips = list(subnet)[1:]

try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    net = slice.add_l2network(name=network_name, subnet=subnet)
    # Node1
    node1 = slice.add_node(name=node1_name, site=site, image=image)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    iface1.set_mode('auto')
    net.add_interface(iface1)
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site, image=image)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    iface2.set_mode('auto')
    net.add_interface(iface2)
    
    # Node3
    node3 = slice.add_node(name=node3_name, site=site, image=image)
    iface3 = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]
    iface3.set_mode('auto')
    net.add_interface(iface3)

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


Retry: 8, Time: 263 sec


ID,cc940914-adb8-4cb8-9730-7d7ad5e6bc20
Name,DHCPSlice
Lease Expiration (UTC),2023-04-12 04:14:25 +0000
Lease Start (UTC),2023-04-11 04:14:26 +0000
Project ID,7bc05490-ba40-45ba-99dc-b011680825d2
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
4ae1fdcd-7d97-42ef-9f44-62bb9031d88b,Server,2,8,10,default_ubuntu_22,qcow2,clem-w3.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fea1:9718,Active,,ssh ${Username}@${Management IP},/Users/devinlane/Documents/fabric/src/fabric_config/slice_key.pub,/Users/devinlane/Documents/fabric/src/fabric_config/slice_key
efb1a501-5573-4a75-a544-b4d4348c986d,Client1,2,8,10,default_ubuntu_22,qcow2,clem-w3.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fe98:50aa,Active,,ssh ${Username}@${Management IP},/Users/devinlane/Documents/fabric/src/fabric_config/slice_key.pub,/Users/devinlane/Documents/fabric/src/fabric_config/slice_key
823cd95e-4dbb-4bc2-97bb-70f6b0b35758,Client2,2,8,10,default_ubuntu_22,qcow2,clem-w3.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fef5:1a2b,Active,,ssh ${Username}@${Management IP},/Users/devinlane/Documents/fabric/src/fabric_config/slice_key.pub,/Users/devinlane/Documents/fabric/src/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
b17ab83d-830a-4a9e-94d7-e651d8acc4b5,DHCP-demo-net,L2,L2Bridge,CLEM,10.0.0.0/24,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address
Server-nic1-p1,p1,Server,DHCP-demo-net,100,auto,,06:49:81:02:94:C9,ens7,ens7,10.0.0.3
Client1-nic2-p1,p1,Client1,DHCP-demo-net,100,auto,,06:A4:81:4A:F5:D3,ens7,ens7,10.0.0.2
Client2-nic3-p1,p1,Client2,DHCP-demo-net,100,auto,,0A:B0:19:DF:54:85,ens7,ens7,10.0.0.1



Time to print interfaces 270 seconds


### Threading Function to Speed up Multi-Node Actions:

In [5]:
def thread_ripper(command):
    nodes = slice.get_nodes()
    threads = {}
    for node in nodes:
        print(f'Create thread for node: {node.get_name()}')
        threads[node] = node.execute_thread(command, output_file=f'logs/{node.get_name()}.log')

    print('Done creating threads!')

    for node, thread in threads.items():
        print(f'node: {node.get_name()}... ', end='')
        stdout, stderr = thread.result()
        print('done!')

## Install DHCP Server to the Server Node

You can tail the log files for each machine with the following command:
```bash
tail -f logs/{node name}.log
```

In [6]:
# Get nodes
# What we will place the DHCP server on
server = slice.get_node(name=node1_name)
# Our DHCP Clients
client1 = slice.get_node(name=node2_name)
client2 = slice.get_node(name=node3_name)

# Step 1: Update packages:
thread_ripper('sudo apt-get update && sudo apt-get upgrade -y')
thread_ripper('sudo apt-get install -y net-tools network-manager')

Create thread for node: Server
Create thread for node: Client1
Create thread for node: Client2
Done creating threads!
node: Server... done!
node: Client1... done!
node: Client2... done!
Create thread for node: Server
Create thread for node: Client1
Create thread for node: Client2
Done creating threads!
node: Server... done!
node: Client1... done!
node: Client2... done!


### Configure our DHCP network:

In [7]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

In [8]:
# Step 1: Give the DHCP server an address on the same DHCP subnet we want
try:
    server_iface = server.get_interface(network_name=network_name) 
    server_addr = available_ips.pop(0)
    server_iface.ip_addr_add(addr=server_addr, subnet=subnet)

    stdout, stderr = server.execute(f'sudo ifconfig {server_iface.get_os_interface()} up')
    stdout, stderr = server.execute(f'ip addr show {server_iface.get_os_interface()}')
except Exception as e:
    print(f"Exception: {e}")

3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 06:49:81:02:94:c9 brd ff:ff:ff:ff:ff:ff
    altname enp0s7
    inet 10.0.0.3/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet 192.168.1.1/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::449:81ff:fe02:94c9/64 scope link tentative 
       valid_lft forever preferred_lft forever


In [9]:
# Step 2: Install DHCP Server
stdout, stderr = server.execute('sudo apt-get install -y isc-dhcp-server', quiet=True, output_file=f'logs/server.log')
# server.execute('sudo rm /etc/dhcp/dhcpd.conf && sudo dpkg --configure -a') # Run this if the above fails

In [10]:
# Step 3: Configure server
stdout, stderr = server.execute('''sudo bash -c 'echo "default-lease-time 600;
max-lease-time 7200;
authoritative;
 
subnet 192.168.1.0 netmask 255.255.255.0 {
 range 192.168.1.100 192.168.1.200;
 option subnet-mask 255.255.255.0;
}
" >> /etc/dhcp/dhcpd.conf'
''')
server.execute(r'''sudo sed -i 's/INTERFACESv4=""/INTERFACESv4="ens7"/' /etc/default/isc-dhcp-server''', quiet=True, output_file='logs/server.log');

In [11]:
# Step 4: Restart the server
server.execute('sudo systemctl restart isc-dhcp-server.service');
server.execute('sudo systemctl status isc-dhcp-server.service');

● isc-dhcp-server.service - ISC DHCP IPv4 server
     Loaded: loaded (/lib/systemd/system/isc-dhcp-server.service; enabled; vendor preset: enabled)
     Active: active (running) since Tue 2023-04-11 04:24:01 UTC; 1s ago
       Docs: man:dhcpd(8)
   Main PID: 18747 (dhcpd)
      Tasks: 4 (limit: 9493)
     Memory: 4.5M
        CPU: 8ms
     CGroup: /system.slice/isc-dhcp-server.service
             └─18747 dhcpd -user dhcpd -group dhcpd -f -4 -pf /run/dhcp-server/dhcpd.pid -cf /etc/dhcp/dhcpd.conf ens7

Apr 11 04:24:01 Server dhcpd[18747]: PID file: /run/dhcp-server/dhcpd.pid
Apr 11 04:24:01 Server dhcpd[18747]: Wrote 0 leases to leases file.
Apr 11 04:24:01 Server sh[18747]: Wrote 0 leases to leases file.
Apr 11 04:24:01 Server dhcpd[18747]: Listening on LPF/ens7/06:49:81:02:94:c9/192.168.1.0/24
Apr 11 04:24:01 Server sh[18747]: Listening on LPF/ens7/06:49:81:02:94:c9/192.168.1.0/24
Apr 11 04:24:01 Server sh[18747]: Sending on   LPF/ens7/06:49:81:02:94:c9/192.168.1.0/24
Apr 11 04:24:01

## Request IP Addresses (Client-Side)

In [12]:
clients = {}
for client in (client1, client2,):
    # Request DHCP Address
    client.execute('sudo dhclient ens7');
    # Extract new IP addresses for later use (ping)
    stdout, stderr = client.execute('''ip addr show ens7 | grep "inet " | awk '{print $2}' | cut -d/ -f1''', quiet=True);
    clients[client.get_name() + '_address'] = stdout[:-1].split('\n')[1] # remove newline
    
clients

{'Client1_address': '192.168.1.100', 'Client2_address': '192.168.1.101'}

## Run the Experiment

We will ping client 2 using its newly allocated DHCP address.


In [13]:
try:        
    stdout, stderr = client1.execute(f"ping -c 5 {clients['Client2_address']}")
    
except Exception as e:
    print(f"Exception: {e}")

PING 192.168.1.101 (192.168.1.101) 56(84) bytes of data.
64 bytes from 192.168.1.101: icmp_seq=1 ttl=64 time=0.212 ms
64 bytes from 192.168.1.101: icmp_seq=2 ttl=64 time=0.134 ms
64 bytes from 192.168.1.101: icmp_seq=3 ttl=64 time=0.135 ms
64 bytes from 192.168.1.101: icmp_seq=4 ttl=64 time=0.156 ms
64 bytes from 192.168.1.101: icmp_seq=5 ttl=64 time=0.122 ms

--- 192.168.1.101 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4084ms
rtt min/avg/max/mdev = 0.122/0.151/0.212/0.032 ms


## Delete the Slice

Please delete your slice when you are done with your experiment.

In [14]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")